# Introduction
This machine learning project, is aim to train a model that could detect whether a person in an uploaded image is wearing a mask or not. I collected and trained the model using a dataset of images of people, some with masks and some without. The model learned to distinguish masked and unmasked faces, enabling real-time prediction of mask status when new images are provided.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import Libaries

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

Learning

In [ ]:
# Set image size and batch size
image_size = (128, 128)
batch_size = 32

# Prepare the data generator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_data = datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

# Save the model
model.save("mask_detector_model.h5")
print("Model saved as mask_detector_model.h5")



Plot training & validation accuracy and loss values



In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

PREDICT ON A NEW IMAGE

In [ ]:
# Load the saved model
model = tf.keras.models.load_model("mask_detector_model.h5")

# Load and preprocess a single image
img_path = "/content/drive/MyDrive/gil.jpg"
img = image.load_img(img_path, target_size=(128, 128))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

Prediction Result

In [ ]:
prediction = model.predict(img_array)

# Interpret result (binary classification)
if prediction[0][0] < 0.5:
    print("Prediction: WITH MASK")
else:
    print("Prediction: WITHOUT MASK")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Prediction: WITH MASK


# Conclusion
The model trained can reliably detect whether an image has a masked or unmasked face. Overall, the project successfully demonstrated how deep learning can be applied to real-world problems of public health and safety.